In [1]:
from QHyper.solvers.quantum_annealing.advantage import Advantage

import numpy as np
import networkx as nx
from collections import defaultdict

from QHyper.converter import Converter
from QHyper.constraint import Polynomial

from dwave.system import DWaveSampler, EmbeddingComposite
from dimod import BinaryQuadraticModel
from dimod.sampleset import SampleSet

Some util function from QHyper

In [2]:
def convert_qubo_keys(qubo: Polynomial) -> tuple[dict[tuple, float], float]:
    new_qubo = defaultdict(float)
    offset = 0.0

    qubo, offset = qubo.separate_const()
    for k, v in qubo.terms.items():
        if len(k) == 1:
            new_key = (k[0], k[0])
        elif len(k) > 2:
            raise ValueError("Only supports quadratic model")
        else:
            new_key = k

        new_qubo[new_key] += v

    return (new_qubo, offset)

Load our biggest graph

In [3]:
Graphs = np.load("networks/powerlaw_m=1_p=0.2/graphs.npy", allow_pickle=True)
G = Graphs[9]

Simple binary clustering of this graph

In [4]:
from QHyper.problems.community_detection import CommunityDetectionProblem, Network


network = Network(graph=G)
problem = CommunityDetectionProblem(network_data=network, communities=2, one_hot_encoding=False)

Get the standard Advantage sampler and embedding composite

In [5]:
sampler = DWaveSampler(solver="Advantage_system5.4", region="eu-central-1")
embedding_composite = EmbeddingComposite(sampler)

In [6]:
qubo = Converter.create_qubo(problem, [])
qubo_terms, offset = convert_qubo_keys(qubo)
bqm = BinaryQuadraticModel.from_qubo(qubo_terms, offset=offset)

In [8]:
%time sampleset = EmbeddingComposite(sampler).sample(bqm, num_reads=100)

CPU times: total: 1min 27s
Wall time: 1min 58s


In [9]:
res = sampleset.first.sample

c0 = sorted([int(k[1:]) for k, v in res.items() if v == 0])
c1 = sorted([int(k[1:]) for k, v in res.items() if v == 1])

# Some simple intuition on the results
print("c0 len:", len(c0))
print("c1 len:", len(c1))
print("Q: ", nx.community.modularity(G, [c0, c1]))

c0 len: 48
c1 len: 52
Q:  0.4583205795327008


Each time the .sample method is called on the sampler, a new embedding is calculated.\
Previously calculated embeddings can be extracted from the sampleset info.

In [19]:
sampleset = embedding_composite.sample(
    bqm, num_reads=100, return_embedding=True
)

In [20]:
embedding_from_sampleset = sampleset.info["embedding_context"]["embedding"]
embedding_from_sampleset

{'x1': (3604, 3603, 3602, 918, 917, 916, 3605, 3606, 3607, 3608, 915, 3601),
 'x0': (1296,
  1295,
  4057,
  1294,
  3682,
  1353,
  1487,
  1352,
  3427,
  1486,
  1485,
  1297,
  4311,
  4310,
  4309,
  4308,
  4307,
  862),
 'x2': (3440,
  3439,
  3438,
  3437,
  1127,
  3444,
  3441,
  1126,
  3436,
  3442,
  3443,
  3435,
  1128,
  1125),
 'x3': (1163,
  4475,
  3935,
  1162,
  1161,
  1160,
  1159,
  1158,
  1157,
  1156,
  1155,
  4640,
  4639,
  4638,
  909,
  4535,
  4534,
  4533,
  729,
  4532,
  518),
 'x4': (3275, 3276, 3274, 3273, 3277, 3278, 3272, 3279, 3271, 706, 3270, 705),
 'x5': (3931,
  425,
  350,
  349,
  440,
  3995,
  439,
  438,
  3932,
  635,
  3993,
  3994,
  348,
  347,
  346,
  426,
  3996,
  345,
  3997),
 'x6': (1009, 3904, 1010, 1008, 1007, 1006, 1005, 1011, 1012, 1013, 3903),
 'x7': (3887,
  395,
  394,
  3888,
  3889,
  3890,
  3891,
  3892,
  3893,
  396,
  393,
  392,
  391,
  390),
 'x8': (1309,
  3757,
  1308,
  1307,
  3756,
  3755,
  3754,
  3753,

Let's find embedding the same way it is done automatically in EmbeddingComposite class

In [11]:
import minorminer
import dimod


find_embedding=minorminer.find_embedding
child_structure_search = dimod.child_structure_dfs

In [12]:
# apply the embedding to the given problem to map it to the child sampler
target_structure = child_structure_search(sampler)
__, target_edgelist, target_adjacency = target_structure

# add self-loops to edgelist to handle singleton variables
source_edgelist = list(bqm.quadratic) + [(v, v) for v in bqm.linear]

In [14]:
%time embedding_calculated = find_embedding(source_edgelist, target_edgelist)

CPU times: total: 16.9 s
Wall time: 20.4 s


In [21]:
embedding_calculated["x3"]

[3079, 3078, 3077, 3076, 3080, 3081, 3082, 3075, 3083]

In [22]:
embedding_from_sampleset["x3"]

(1163,
 4475,
 3935,
 1162,
 1161,
 1160,
 1159,
 1158,
 1157,
 1156,
 1155,
 4640,
 4639,
 4638,
 909,
 4535,
 4534,
 4533,
 729,
 4532,
 518)

The minorminer.find_embedding default method is heuristic, \
I'll set a seed to see if the embedding we calculate manually is same as the embedding calculated automatically with EmbeddingComposite \

In [23]:
sampleset_with_random_seed = embedding_composite.sample(
    bqm, num_reads=100, return_embedding=True, embedding_parameters={"random_seed": 10}
)

In [24]:
embedding_calculated_rs = find_embedding(source_edgelist, target_edgelist, random_seed=10)

In [25]:
embedding_from_sampleset_rs = sampleset_with_random_seed.info["embedding_context"]["embedding"]

No surprise, they're the same:

In [26]:
sorted(embedding_calculated_rs) == sorted(embedding_from_sampleset_rs)

True

Let's put our manually calulated embedding to FixedEmbeddingComposite

In [27]:
from dwave.system.composites import FixedEmbeddingComposite


fixed_embedding_composite = FixedEmbeddingComposite(sampler, embedding=embedding_calculated_rs)

Let's sample from it

In [29]:
%time sampleset_fe = fixed_embedding_composite.sample(bqm, num_reads=100, return_embedding=True)

CPU times: total: 46.9 ms
Wall time: 105 ms


In [32]:
res_fe = sampleset_fe.first.sample

# Again, some simple results intuition
c0_fe = sorted([int(k[1:]) for k, v in res_fe.items() if v == 0])
c1_fe = sorted([int(k[1:]) for k, v in res_fe.items() if v == 1])

print("c0_fe len:", len(c0_fe))
print("c1_fe len:", len(c1_fe))
print("Sample modularity from EmbeddingComposite:", nx.community.modularity(G, [c0, c1]))
print("Sample modularity from FixedEmbeddingComposite:", nx.community.modularity(G, [c0_fe, c1_fe]))

c0_fe len: 50
c1_fe len: 50
Sample modularity from EmbeddingComposite: 0.4583205795327008
Sample modularity from FixedEmbeddingComposite: 0.4393429241914091


Let's see how much time can be saved when we parallelize minorminer.find_embedding

In [33]:
from joblib import Parallel, delayed


N_RUNS = 8
n_jobs = 8

%time embeddings = Parallel(n_jobs=n_jobs)(delayed(find_embedding)(source_edgelist, target_edgelist, random_seed=10) for _ in range(N_RUNS))

CPU times: total: 93.8 ms
Wall time: 3min 5s


In [35]:
def run_standard_loop():
    embeddings_2 = []

    for _ in range(N_RUNS):
        emb = find_embedding(source_edgelist, target_edgelist, random_seed=10)
        embeddings_2.append(emb)

%time run_standard_loop()

CPU times: total: 8min 11s
Wall time: 10min 4s


Ineed some time can be saved by paralellizing embedding calculations.

Let's now imagine we go down in hierarchy (hierarchical search) \
and see how time needed to find embedding scales with subgraphs of smaller sizes.\
Let's divide c0:

In [36]:
problem_c0 = CommunityDetectionProblem(network_data=Network(graph=G, community=c0), communities=2, one_hot_encoding=False)
qubo_c0 = Converter.create_qubo(problem_c0, [])
qubo_terms_c0, offset_c0 = convert_qubo_keys(qubo_c0)
bqm_c0 = BinaryQuadraticModel.from_qubo(qubo_terms_c0, offset=offset_c0)

In [37]:
source_edgelist_c0 = list(bqm_c0.quadratic) + [(v, v) for v in bqm_c0.linear]
%time embeddings_c0 = Parallel(n_jobs=n_jobs)(delayed(find_embedding)(source_edgelist_c0, target_edgelist, random_seed=10) for _ in range(N_RUNS))

CPU times: total: 203 ms
Wall time: 8.2 s


In [38]:
def run_standard_loop():
    embeddings_2_c0 = []
    for _ in range(N_RUNS):
        emb = find_embedding(source_edgelist_c0, target_edgelist, random_seed=10)
        embeddings_2_c0.append(emb)

%time run_standard_loop()

CPU times: total: 15.8 s
Wall time: 28.9 s


Not as much time was saved now (c0 of approx. 50 nodes) as with the whole graph (of 100 nodes).

_____________

___________

Last example to show sampling with a fixed embedding is quick (bqm for the full graph of 100 nodes):

In [39]:
def sample_fe():
    sample = FixedEmbeddingComposite(sampler, embedding=embedding_calculated_rs).sample(bqm, num_reads=100)
    return sample.first.sample

def sample_fe_loop():
    results = []
    for _ in range(N_RUNS):
        results.append(sample_fe())
    # return results

%time sample_fe_loop()

CPU times: total: 1.12 s
Wall time: 5.51 s


_____________

_____________

Comparison with dwave.embedding.pegasus find_clique_embedding

In [8]:
from dwave.embedding.pegasus import find_clique_embedding
import dwave_networkx as dnx

In [9]:
bqm_graph = bqm.to_networkx_graph()

C:\Users\basia\AppData\Local\Temp\ipykernel_22728\3242365722.py:1: DeprecationWarning: BinaryQuadraticModel.to_networkx_graph() is deprecated since dimod 0.10.0 and will be removed in 0.12.0. Use dimod.to_networkx_graph() instead.
  bqm_graph = bqm.to_networkx_graph()


In [10]:
from minorminer.busclique import busgraph_cache

busgraph_cache.clear_all_caches()

In [11]:
%time emb = find_clique_embedding(bqm_graph, target_graph=sampler.to_networkx_graph())

CPU times: total: 1min 41s
Wall time: 2min 24s


Embeddings already calculated are cached with busgraph_cache:

In [12]:
%time emb = find_clique_embedding(bqm_graph, target_graph=sampler.to_networkx_graph())

CPU times: total: 31.2 ms
Wall time: 85.5 ms


In [17]:
from dwave.system.composites import FixedEmbeddingComposite


sample_cl = FixedEmbeddingComposite(sampler, embedding=emb).sample(bqm, num_reads=100)

res_cl = sample_cl.first.sample

c0_cl = sorted([int(k[1:]) for k, v in res_cl.items() if v == 0])
c1_cl = sorted([int(k[1:]) for k, v in res_cl.items() if v == 1])

print("c0_cl len:", len(c0_cl))
print("c1_cl len:", len(c1_cl))
# print("Sample modularity from EmbeddingComposite:", nx.community.modularity(G, [c0, c1]))
print("Sample modularity from FixedEmbeddingComposite:", nx.community.modularity(G, [c0_cl, c1_cl]))

c0_cl len: 49
c1_cl len: 51
Sample modularity from FixedEmbeddingComposite: 0.4696969696969697
